# Overview
This notebook is to develop Twitter Sentiment Analysis

__Date__:  
2021-12-20

## Useful Notes
1. Github:  
https://github.com/twitterdev/Twitter-API-v2-sample-code  
2. API command:  
curl --request GET 'https://api.twitter.com/2/tweets/search/recent?query=from:twitterdev' --header 'Authorization: Bearer $BEARER_TOKEN'
3. Beginner Guide:  
beginner guide https://www.dataquest.io/blog/python-api-tutorial/
4. Get Requests Params:  
-url: enpoint that you want to request  
-auth: authentication if it is needed  
-params: queryparams (still trying to find out)  
5. Tweepy Sample Code:  
https://dev.to/twitterdev/a-comprehensive-guide-for-using-the-twitter-api-v2-using-tweepy-in-python-15d9
6. Kill fast api
https://stackoverflow.com/questions/64588486/address-already-in-use-fastapi
7. Permission to use AWS Comprehend with EC2 instance
-create an IAM role for that instance, and add comprehend policies to the role 
8. Connecting EC2 with Cloud9 IDE
https://towardsdatascience.com/creating-aws-ec2-and-connecting-it-with-aws-cloud9-ide-and-aws-s3-a6313aa82ec#eedd
9. SSH to remote server with VSC
https://www.youtube.com/watch?v=7kum46SFIaY


# Main Goal:

1) Understand types of API
2) Rest API (and other type?)
3) Postman
4) What to expect when talking about API
5) Check the API provided by AIE team
6) Try create my own API
7) User tweet timeline vs Recent Search
8) Advanced filter for recent search

#doubt:
1) os
2) why videos one cant work, github one can (request.get)
3) what is response returning (use dir(response) to list them out) / check this link https://www.geeksforgeeks.org/response-methods-python-requests/
4) vsc terminal which python is different path
5) @decorator, classess
6) why http://backend cant word
7) what does :8080 means
8) host vs port

Project Pending:
1) next step, clean code and env
2) make a sample visualization
3) webapp
4) try with reply?

5) useful field, reference tweets, conversational id
6) how i want my df to be
7) what thigns do i need
8) speed improvement, use more dict / json , output df only at the last step
9) -Pip vs pip3 
10) AWS Iamrole, Policies, IAM users, how to check permissions
11) python env, python3 .py, python.py, conda .py
12) IDE for EC2 instance

In [131]:
import pandas as pd
import requests
import os
import json
import tweepy
from configparser import ConfigParser
import boto3

parser = ConfigParser()
parser.read('api_auth.cfg')

#This is the super secret information that will allow access to the Twitter API and AWS Comprehend
bearer_token = parser.get('api_tracker', 'bearer_token')
aws_key_id =  parser.get('api_tracker', 'aws_key_id')
aws_key =  parser.get('api_tracker', 'aws_key')

In [134]:
def search_recent_tweets_as_df(query, tweet_fields):
    client = tweepy.Client(bearer_token=bearer_token)
    tweets = client.search_recent_tweets(query=query, tweet_fields=tweet_fields, max_results=100)
    df=pd.DataFrame(tweets.data)
    
    return df

In [146]:
def text_sentiment(df):

    client = boto3.client('comprehend',  region_name='us-west-2')

    sentiment_list = []
    sentiment_score_positive_list = []
    sentiment_score_negative_list = []
    sentiment_score_neutral_list = []
    sentiment_score_mixed_list = []

    for text in df.text:
        #text_data = base64.b64decode(text).decode('utf-8').strip()
        sentiment_all = client.detect_sentiment(Text=text, LanguageCode='en')
        sentiment_list.append(sentiment_all['Sentiment'])
        sentiment_score_positive_list.append(sentiment_all['SentimentScore']['Positive'])
        sentiment_score_negative_list.append(sentiment_all['SentimentScore']['Negative'])
        sentiment_score_neutral_list.append(sentiment_all['SentimentScore']['Neutral'])
        sentiment_score_mixed_list.append(sentiment_all['SentimentScore']['Mixed'])

    df['sentiment'] = sentiment_list
    df['sentiment_score_positive'] = sentiment_score_positive_list
    df['sentiment_score_negative'] = sentiment_score_negative_list
    df['sentiment_score_mixed'] = sentiment_score_mixed_list

    return df

In [ ]:
query = 'from:Bitcoin -is:retweet' #get reply, 'conversation_id:1471510774505893906'
tweet_fields = ['context_annotations', 'created_at','conversation_id']

df = search_recent_tweets_as_df(query, tweet_fields)
df_sentiment = text_sentiment(df)
df_sentiment.to_csv('tweets_with_sentiment.csv')

print(df_sentiment.head())

## Output Visualization

In [140]:
df_sentiment = pd.read_csv('tweets_with_sentiment.csv')
df_sentiment

,context_annotations,conversation_id,created_at,id,text,sentiment,sentiment_score_positive,sentiment_score_negative,sentiment_score_mixed
0,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",1473692922323386380,2021-12-22 16:32:27+00:00,1473692922323386380,Only you #Bitcoin buyers can prevent red candl...,NEUTRAL,0.039692,0.047798,0.006594
1,"[{'domain': {'id': '10', 'name': 'Person', 'de...",1473691438424723463,2021-12-22 16:26:33+00:00,1473691438424723463,“It might make sense just to get some in case ...,NEUTRAL,0.080573,0.236311,0.000940
2,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",1473419222562660355,2021-12-21 22:24:52+00:00,1473419222562660355,Financial companies &amp; central banks have w...,NEGATIVE,0.003547,0.534025,0.038486
3,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",1473414095567130626,2021-12-21 22:04:29+00:00,1473414095567130626,Is this what $98k Nov and $135k Dec #Bitcoin l...,NEUTRAL,0.006312,0.016177,0.000051
4,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",1473063792493809673,2021-12-20 22:52:30+00:00,1473063792493809673,There are around 4m #Bitcoin hodled at prices ...,NEUTRAL,0.021628,0.048044,0.029903
5,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",1472941455756857344,2021-12-20 14:46:23+00:00,1472941455756857344,No one uses #Bitcoin... https://t.co/gEf2j0x3Co,NEUTRAL,0.005693,0.418330,0.032112
6,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",1472941382499115012,2021-12-20 14:46:06+00:00,1472941382499115012,https://t.co/v0bw5t2heO,NEUTRAL,0.008507,0.002727,0.014681
7,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",1472798967624716288,2021-12-20 05:20:11+00:00,1472798967624716288,https://t.co/zEeXbriMRQ,NEUTRAL,0.001258,0.000865,0.000325
8,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",1472277258157002752,2021-12-18 18:47:06+00:00,1472277258157002752,🌊 https://t.co/WJpTWCJs3z,NEUTRAL,0.002698,0.000333,0.000157
9,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",1471986419555917824,2021-12-17 23:31:25+00:00,1471986419555917824,The December 31st #Bitcoin options expiration ...,NEUTRAL,0.028714,0.353425,0.000248


In [142]:
df_sentiment.describe()

,conversation_id,id,sentiment_score_positive,sentiment_score_negative,sentiment_score_mixed
count,1.500000e+01,1.500000e+01,15.000000,15.000000,15.000000
mean,1.472551e+18,1.472562e+18,0.057666,0.113172,0.008296
std,8.286162e+14,8.174591e+14,0.122221,0.180189,0.013718
min,1.471511e+18,1.471511e+18,0.000437,0.000134,0.000007
25%,1.471704e+18,1.471786e+18,0.003122,0.002012,0.000159
50%,1.472799e+18,1.472799e+18,0.007467,0.016177,0.000325
75%,1.473239e+18,1.473239e+18,0.034203,0.142178,0.010638
max,1.473693e+18,1.473693e+18,0.458317,0.534025,0.038486


In [157]:
df_sentiment['sentiment'].value_counts()

NEUTRAL     14
NEGATIVE     1
Name: sentiment, dtype: int64

## Pending